< [How to Read and Represent Data](../ica02/How_to_Read_and_Represent_Data.ipynb) | Contents (TODO) | [Data Preprocessing and Visualization](../ica04/Data_Preprocessing_and_Visualization.ipynb) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica03/Data_Mining.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>

# Data Mining

Data mining is the practice of examining data sources in order to generate new information. People say [data is the new oil](https://www.economist.com/leaders/2017/05/06/the-worlds-most-valuable-resource-is-no-longer-oil-but-data) (* and also [otherwise](https://www.forbes.com/sites/bernardmarr/2018/03/05/heres-why-data-is-not-the-new-oil/#1b2106913aa9)...). So we can arrive at this ~~cheesy~~ analogy that data mining is like mining the oil.

Nonetheless, what we are going to do today in this notebook are threefold. First, we are going to see how to access a large collection of public data sets published on a website called, Kaggle. Second, we will get a feel of web crawling/scraping by extracting some living information on the web. Last but not the least, we are going to see how data APIs generally work.

## 1. Kaggle Datasets

In short, Kaggle is a sort of online community for data scientists, now owned by Google (as of 2017). The community allows its users (called Kagglers) to publish data sets, build models in a web-based data-science environment, enter data science challenge competitions, exchange ideas/codes, etc. Around the time Google acquired Kaggle, its user base passed 1,000,000 registered users, spanning 190+ countries, forming the largest and most diverse data science community in the world.

Especially for students of data science, it is an extremely useful source of materials, not just because of the gigantic amount of real-world data sets and data science problems, but also because of the source codes, ideas, etc. shared by other Kagglers. In this section, I'll show one of (many) ways to use Kaggle for your own project.


### 1.1. Getting Started

#### Sign up
Well, first off, you have to sign up (again, I'm not getting paid by Google for encouraging you to sign up :)). The sign up process is rather simple, so I won't explain here. Once the sign up process is complete, just go ahead and navigate what they have on the website. Especially, try to click `Competitions` menu on top, and see the list of competitions. Also, make sure to check out `Datasets` menu, where you can find tens of thousands of real-world data set for free. Lastly, `Notebooks` can be a great place for your future projects, to get an idea of how to solve a specific problem, etc. 

#### Create an API Token
Now, go to `My Account` by clicking the user profile at the top right corner. 
![User settings menu](figures/kaggle_my_account.png)

Scroll down a bit and you will find an API section. Click '`Create New API Token`' button.
![Create New Kaggle API Token](figures/create_new_kaggle_api_token.png)

This will download a file named `kaggle.json`. Open it with a text editor. You will see something like this in the file.
```json
{"username":"bigdata","key":"0123456789abcdefghijklmn"}
```
Make sure to keep it opened somewhere as you will need it a few cells later.

### 1.2. Install Python Kaggle Library

Now, in order to use Kaggle on your local machine (or Google Colab), you will need to install python Kaggle library, which can be done simply by one line of code.

In [ ]:
# This is how you install a python library
!pip install kaggle

Now, the next thing is to provide the Kaggle API token to the library we just installed so that it knows you are a legitimate Kaggler. To this, copy and past the contents of the json file you downloaded above in the cell below.

In [ ]:
# type your Kaggle API token here
token = {"username":"bigdata","key":"0123456789abcdefghijklmn"}

Now the code below is for advanced users only. You don't have to understand them line by line. Just run the cell and safely ignore what's in it. Just keep in mind in the future that there was something about the API token and you needed to set it up. When you need to do it again in the future, just come back to this notebook and copy the code and reuse it.

In [ ]:
# This cell is only for advanced users. Run this cell and you can safely move on to the next cell
import os
from pathlib import Path
import json
import platform

# creates and places the token file at a desired location
home = str(Path.home())
kaggle_root = os.path.join(*[home, '.kaggle'])
os.mkdir(kaggle_root)
with open(os.path.join(*[kaggle_root, 'kaggle.json']), 'w') as file:
    json.dump(token, file)

# make the key file accessible only to the owner
if platform.system() == 'Windows':
    !attrib -R {os.path.join(*[kaggle_root, 'kaggle.json'])}
else:
    !chmod 600 {os.path.join(*[kaggle_root, 'kaggle.json'])}

### Alternative Method (Colab Only)

If you are using Colab, you can upload the json file using the file dialog below.

In [ ]:
from google.colab import files
uploaded = files.upload()

Then, run the cell below.

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/kaggle.json
!chmod 600 ~/.kaggle/kaggle.json

### 1.3. Downloading a Data Set from Kaggle

Downloading a data set from kaggle is as simple as just one line.
```bash
!kaggle datasets download -d <path-to-dataset> -p <download-location>
```

Path to data set is what comes after `http://www.kaggle.com/` in the data set URL. For example, somebody gathered avocado prices and published as a data set in kaggle, which can be found at https://www.kaggle.com/neuromusic/avocado-prices. So In this case, the path to data set is `neuromusic/avocado-prices`.

Download location means the name of the folder on your hard drive (or Google virtual machine's hard drive, if you're using Colab). Say, you want to create a folder called `data` under the present working directory (where this notebook `ipynb` file is located). You can simply type something like below to download the avocado data set under the said folder.

In [ ]:
!kaggle datasets download -d neuromusic/avocado-prices -p data

Many of the data sets in Kaggle come with some sort of a compressed file (e.g. zip file in this case). For Mac/Linux users, you can simply type in a code cell:
```bash
!unzip ./data/avocado-prices.zip -d ./data
```
to extract all the contents of the zip file into the data folder. If you are a Windows user, however, the story is a bit different. You will in fact have to unzip the file manually by opening up the 'File Explorer' and tracking down to the folder that you downloaded the dataset. If you have 7-zip or other Windows unzip tools installed already, or if you have Java Development Kit installed, there are ways to achieve this conveniently in a notebook, without having to do everything manually (see [this](https://stackoverflow.com/questions/1021557/how-to-unzip-a-file-using-the-command-line) for detail). However, certainly, this is beyond the scope of this class, so I won't dive too much into details. 

In [ ]:
# Unzipping files.
if platform.system() == 'Windows':
    this_file_path = !echo %cd%
    data_path = os.path.join(*[this_file_path[0], 'data'])
    print('[IMPORTANT] No automatic unzipping supported on Windows.')
    print('You have to open `File Explorer` and manually unzip `' + data_path + '\\avocado-prices.zip`')
    print('Make sure `avocado.csv` file in the zip file is placed directly under `data` folder:')
    print('|- ica03')
    print('    |- data')
    print('        |- avocado.csv')
    print('    |- Data_Mining.ipynb')
else:
    !unzip ./data/avocado-prices.zip -d ./data

Okay, now you are ready to play with the avocado data. The avocado data set comes with a single comma seprated values (CSV) file.

### 1.4. Assignment

Find a data set on Kaggle with a search keyword "iowa". Read the downloaded data set as Spark DataFrame. Display summary statistics.

## 2. Web Scraping with BeautifulSoup

BeautifulSoup is a python library that comes with a lot of handy functions for web scraping and gathering information from the internet. There are so many things you can do with BeutifulSoup, but in this notebook, I'll show you a rather specific example of how BeutifulSoup can be applied for data mining.

To this, we will use the Eastern Iowa - Cedar Rapids Airport website as an example. There, they provide a real-time flight status update for travellers (https://flycid.com/flight-status/). Let's click and open this website and see how it look like.

![Cedar Rapids Airport Webpage](figures/cid_web.png)

### 2.1. Anatomy of a Web Page

Different people would have different approaches, but what I usually do is to take a look at the anatomy of the web page using my web browser's developer tool. If you use Chrome or Firefox, the developer tool can be opened by pressing `ctrl (cmd) + shift + I` or `F12`. If you use Safari, it is called Web Inspector, and can be opened with `cmd + shift + I`. For other web browsers, there should be a menu somewhere, or an instruction on the internet.

Now, in the developer tool, you should find some scripts which define the web page. In Chrome, it looks like this:

![Developer Tools in Chrome](figures/dev_tools.png)

The script here looks a lot like XML we learned in the [previous lecture](https://docs.google.com/presentation/d/17HzZmXP-xWtvgPrPOptM-AEKFnGaUJSzmEiJjz784_c/edit?usp=sharing). It is in fact called Hypertext Markup Language, or HTML, which is a standard markup language for web documents. You don't have to know all the tags of HTML. However, if you are curious about some basic HTML tags, here's a [nice summary of most commonly used HTML tags](https://www.geeksforgeeks.org/most-commonly-used-tags-in-html/).

Now, most web browsers highlights a specific part of web document when you hover a mouse cursor over a script in the developer tool, like in the screenshot below.

![Developer Tools can Highlight Areas](figures/code_highlight.png)

This is where your job as a data scientist gets less elegant but a little dirty and brute force (welcome to the real world!): The first thing to do to extract an information from a web document is to figure out exactly where the desired information is located. In this example, after a few minutes of digging in (basically hovering the mouse cursor on different locations of the HTML scripts), I found that the flight information was being displayed as an `iframe`, which is basically like a web page within a web page.

![Actual Flight Information in iframe](figures/iframe.png)

What this means is that the airport website is not actually doing anything by itself to retrieve the flight information, but instead, displays an external web page (https://webservice.prodigiq.com/wfids/CID/small?rows=18) within the airport web page as if it is a part of the web page. Long story short, this is where all the desired information we need and, hence, where we will do the web scraping.

### 2.2. Get and Parse HTML

Now that we know where the information exists, let's retrieve the HTML tags and parse them into a useful information for us. First off, let's retrieve the entire web page.

In [ ]:
import requests
page = requests.get("https://webservice.prodigiq.com/wfids/CID/small?rows=18")

Now, with BeutifulSoup, we parse the information and display it in the notebook.

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

In [ ]:
print(soup.prettify())

There are a lot of things going on, but after another dirty work of digging into the tags, we can find the flight information table lives in the tag `table` with an attribute `class="views-table cols-5"`, which can be searched by BeutifulSoup:

In [ ]:
table = soup.find('table', {'class': 'views-table cols-5'})
print(table)

Furthermore, within the table, it seems like all the flight information is structured within `tbody` tag.

In [ ]:
tbody = table.find('tbody')
tbody

For further break down, each flight now is within `tr` task. So, we are going to find all `tr` tags in `tbody` and create a list. Just as a crash HTML course, `tr` is an abbreviation for 'table row' while `td` is for 'table data (column)'. 

In [ ]:
trows = tbody.find_all('tr')
trows[0]

As we can see, each table row (`tr`) contains multiple table data (`td`). In this case, the first (counting from zero) `td` tag contains the flight number, the second contains the city of departure, the third the arrival time, the fourth baggage claim, and last the arrival status:

In [ ]:
print('{:10s} | {:15s} | {:15s} | {:15s} | {:10s}'.format(
    'Flight', 'Departure City', 'Arrival Time', 'Baggage Claim', 'Status'))
for i, trow in enumerate(trows):
    titems = trow.find_all('td')   # find all the data items in each row
    print('{:10s} | {:15s} | {:15s} | {:15s} | {:10s}'.format(
        titems[1].contents[0],          # contents of the first table item (column)
        titems[2].contents[0],          # contents of the second table item
        titems[3].contents[0],          # contents of the third table item
        titems[4].contents[0],          # contents of the fourth table item
        titems[5].contents[0],          # contents of the fifth table item
    ))

There, now you can retrieve the real-time flight arrival information at the Cedar Rapids Airport!

### 2.3. Assignment: College of Engineering Faculty-Staff Directory

Use BeautifulSoup to parse the college of engineering faculty-staff directory from https://www.engineering.uiowa.edu/faculty-staff-directory. Create a Spark DataFrame containing the following columns: `first-name`, `last-name`, `middle-initial`, `title`, `department`, `email`, `campus-address`, and `image-url`. 

## 3. Get Live Stock Price using `yahoo_fin` API

As we have seen above, writing a scraping/crawling code from scratch involves a lot of dirty, brute-force works. For many cases, however, there are people who have already gone through all these and quite generously decided to build a set of handy functions that let you skip all those hassles. Or, sometimes, engineers and developers at companies, who actually built the web pages and knows exactly how the information is structured, decided to provide "nerd users" like me ways to access their data. Whatever the reason was, a data set API is basically a predefind set of functions that helps you access the data.

In this quick example, we are going to retrive real-time stock price data using Yahoo! Finance API (`yahoo_fin`). Let us first install `yahoo_fin` API. 

In [ ]:
!pip install --upgrade yahoo_fin

Now, `yahoo_fin` library comes with lots of modules in it. Among them, in this example, we are going to use `stock_info` module. Importing it should look like this:

In [ ]:
from yahoo_fin import stock_info as si

For data set APIs, we are not going to get too much into details, but here are several things you can do to retrieve real-time stock info.

In [ ]:
# Get Netflix (NFLX) stock info from year 2015 to 2018
data = si.get_data('NFLX', start_date='01/01/2015', end_date='12/31/2018')
data

In [ ]:
data.index # gives time stamps

In [ ]:
data['volume'].values  # gives values of the column named 'volume'

In [ ]:
data[['open','close']].values # gives multiple columns

## 4. Conclusion

We saw a few examples in this notebook of how to gather data from (1) public data repository, (2) manual web scraping, and (3) data set API functions. There are actually a lot more things to cover just for this topic of data mining and what you just saw above are just a tip of the iceberg. But don't worry. As long as you understand the principles above, learning other tools/methods shouldn't be a big challenge, but just a little bit of investment of time to get to know all those commands and etc.

< [How to Read and Represent Data](../ica02/How_to_Read_and_Represent_Data.ipynb) | Contents (TODO) | [Data Preprocessing and Visualization](../ica04/Data_Preprocessing_and_Visualization.ipynb) >

<a href="https://colab.research.google.com/github/stephenbaek/bigdata/blob/master/in-class-assignments/ica03/Data_Mining.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open and Execute in Google Colaboratory"></a>